In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

#### 如果您只希望将代理的最终输出流式传输，则可以使用回调FinalStreamingStdOutCallbackHandler。为此，底层LLM也必须支持流。

In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.callbacks.streaming_stdout_final_only import (
    FinalStreamingStdOutCallbackHandler,
)
from langchain.llms import OpenAI

In [2]:
# 让我们使用streaming = True创建底层LLM，并传递一个新的FinalStreamingStdOutCallbackHandler实例。
llm = OpenAI(
    streaming=True, callbacks=[FinalStreamingStdOutCallbackHandler()], temperature=0
)

In [3]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)
agent.run(
    "It's 2023 now. How many years ago did Konrad Adenauer become Chancellor of Germany."
)

 Konrad Adenauer became Chancellor of Germany in 1949, making it 74 years ago in 2023.

'Konrad Adenauer became Chancellor of Germany in 1949, making it 74 years ago in 2023.'

In [4]:
# 默认情况下，我们假设令牌序列“Final”、“Answer”、“:”表示代理已达成应答。但是，我们也可以传递一个自定义序列作为回答前缀。
llm = OpenAI(
    streaming=True,
    callbacks=[
        FinalStreamingStdOutCallbackHandler(answer_prefix_tokens=["The", "answer", ":"])
    ],
    temperature=0,
)

In [5]:
# 为方便起见，在与answer_prefix_tokens比较时，回调会自动去掉空白和换行字符。例如，如果answer_prefix_tokens = ["The"， " answer"， ":"]，那么["\nThe"， " answer"， ":"]和["The"， " answer"， ":"]将被识别为答案前缀。
from langchain.callbacks.base import BaseCallbackHandler


class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs) -> None:
        # print every token on a new line
        print(f"#{token}#")


llm = OpenAI(streaming=True, callbacks=[MyCallbackHandler()])
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)
agent.run(
    "It's 2023 now. How many years ago did Konrad Adenauer become Chancellor of Germany."
)

# I#
# need#
# to#
# figure#
# out#
# when#
# Kon#
#rad#
# Aden#
#auer#
# became#
# Chancellor#
#
Action#
#:#
# Wikipedia#
#
Action#
# Input#
#:#
# "#
#K#
#on#
#rad#
# Aden#
#auer#
#"#
##
# I#
# need#
# to#
# find#
# out#
# what#
# year#
# Kon#
#rad#
# Aden#
#auer#
# became#
# Chancellor#
#
Action#
#:#
# Wikipedia#
#
Action#
# Input#
#:#
# "#
#K#
#on#
#rad#
# Aden#
#auer#
#"#
##
# I#
# now#
# know#
# the#
# answer#
#
Final#
# Answer#
#:#
# Kon#
#rad#
# Aden#
#auer#
# became#
# Chancellor#
# of#
# Germany#
# in#
# 1949#
#,#
# 74#
# years#
# ago#
#.#
##


'Konrad Adenauer became Chancellor of Germany in 1949, 74 years ago.'

In [6]:
# 当参数stream_prefix = True被设置时，应答前缀本身也将被流式传输。当答案前缀本身是答案的一部分时，这可能很有用。例如，当您的答案是JSON时，如
{ "action": "Final answer", "action_input": "Konrad Adenauer became Chancellor 74 years ago." }
您不仅希望action_input可以流,而且需要整个JSON。

SyntaxError: invalid character '。' (U+3002) (3308388882.py, line 3)